In [ ]:
import pandas as pd
from FATS.Feature import FeatureSpace
from ztfrapid import ztf_rapid
import numpy as np
from astropy.io import fits as pf

In [ ]:
lc_data = pd.read_pickle('/home/nmiranda/workspace/ztf_rapid/data/raw/rcf_marshallc_sncosmo_200114_2018classupdate_addedcv.pkl')

In [ ]:
this_id = list(lc_data.keys())[0]
this_lcs = lc_data[this_id]
this_lc = this_lcs[this_lcs['band'] == 'p48r']
this_mag = ztf_rapid.get_mag(this_lc)
this_mjd = np.array(this_lc[this_lc['flux'] > 0.0]['mjd'])
this_mag_err = ztf_rapid.get_mag_err(this_lc)
fields = [this_mag, this_mjd, this_mag_err]

In [ ]:
feature_list = ['CAR_sigma', 'CAR_mean', 'Meanvariance', 'Mean', 'PercentAmplitude', 'Skew', 'AndersonDarling', 'Std', 'MedianAbsDev', 'Q31', 'Amplitude', 'PeriodLS']

In [ ]:
feature_space = FeatureSpace(featureList=feature_list)
# feature_space = feature_space.calculateFeature(np.array([this_mag, this_mjd, this_mag_err]))
feature_space = feature_space.calculateFeature(np.array(fields))
feature_space.result()

In [ ]:
set(lc_data[this_id]['band'])

In [ ]:
result_list = list()
id_list = list()
target_list = list()
for this_id in lc_data.keys():
    this_lcs = lc_data[this_id]
    if len(this_lcs) < 1:
        continue
    this_lc = this_lcs[this_lcs['band'] == 'p48r']
    this_mag = ztf_rapid.get_mag(this_lc)
    this_mjd = np.array(this_lc[this_lc['flux'] > 0.0]['mjd'])
    this_mag_err = ztf_rapid.get_mag_err(this_lc)
    if len(this_mag) < 1:
        continue
    fields = [this_mag, this_mjd, this_mag_err]
    feature_space = FeatureSpace(featureList=feature_list)
    feature_space = feature_space.calculateFeature(np.array(fields))
    result_list.append(feature_space.result())
    id_list.append(this_id)
    target_list.append(this_lc.meta['classification'])

In [ ]:
features = pd.DataFrame(result_list, columns=feature_list)

In [ ]:
features['ztfid'] = id_list
features['target'] = target_list

In [ ]:
features

In [ ]:
table = features
coldefs = [
    pf.Column(name='ztfid', format='12A', array=np.array(table['ztfid'])),
    pf.Column(name='CAR_mean', format='F', array=table['CAR_mean']),
    pf.Column(name='Meanvariance', format='F', array=table['Meanvariance']),
    pf.Column(name='Mean', format='F', array=table['Mean']),
    pf.Column(name='PercentAmplitude', format='F', array=table['PercentAmplitude']),
    pf.Column(name='Skew', format='F', array=table['Skew']),
    pf.Column(name='AndersonDarling', format='F', array=table['AndersonDarling']),
    pf.Column(name='Std', format='F', array=table['Std']),
    pf.Column(name='MedianAbsDev', format='F', array=table['MedianAbsDev']),
    pf.Column(name='Q31', format='F', array=table['Q31']),
    pf.Column(name='Amplitude', format='F', array=table['Amplitude']),
    pf.Column(name='PeriodLS', format='F', array=table['PeriodLS']),
    pf.Column(name='target', format='16A', array=table['target']),
]

In [ ]:
tbhdu = pf.BinTableHDU.from_columns(coldefs)
tbhdu.writeto('/home/nmiranda/workspace/ztf_rapid/data/interim/rcf_fats_features.fits', checksum=True, overwrite=True)

In [ ]:
ztf_rapid.generate_fits_features_band(
    '/home/nmiranda/workspace/ztf_rapid/data/raw/rcf_marshallc_sncosmo_200114_2018classupdate_addedcv.pkl', 
    '/home/nmiranda/workspace/ztf_rapid/data/interim/rcf_fats_features_p48r.fits', 
    'p48r')

In [ ]:
ztf_rapid.generate_fits_features_band(
    '/home/nmiranda/workspace/ztf_rapid/data/raw/rcf_marshallc_sncosmo_200114_2018classupdate_addedcv.pkl', 
    '/home/nmiranda/workspace/ztf_rapid/data/interim/rcf_fats_features_p48g.fits', 
    'p48g')

In [ ]:
ztf_rapid.generate_fits_features_band(
    '/home/nmiranda/workspace/ztf_rapid/data/raw/rcf_marshallc_sncosmo_200114_2018classupdate_addedcv.pkl', 
    '/home/nmiranda/workspace/ztf_rapid/data/interim/rcf_fats_features_p48i.fits', 
    'p48i')